## 🟢 `Dataset`과 `DataLoader`의 정의  
정해진 형태를 가진다는 것이...  

<br>

### 🟡 **Dataset (데이터셋)**  

- **정의:**  
    - 딥러닝 모델을 학습시키기 위한 **데이터의 집합체**를 의미  
    - **"무엇을"** 학습할 것인가에 대한 **데이터 자체**를 정의.  
    - 단순히 데이터를 모아놓은 것을 넘어, 파이토치(PyTorch)와 같은 딥러닝 프레임워크에서는 `torch.utils.data.Dataset` 클래스를 상속받아 구현하는 것이 일반적  

- `Dataset` 클래스는 다음 두 가지 핵심 메소드를 반드시 포함해야 함  
    1.  `__len__`: 데이터셋에 포함된 전체 샘플(데이터)의 개수를 반환합니다.  
    2.  `__getitem__`: 주어진 인덱스(index)에 해당하는 샘플(데이터)과 그에 맞는 정답(label)을 반환합니다.  

이렇게 구현된 `Dataset`은 모델 학습을 위해 필요한 모든 데이터와 그에 대한 접근 방법을 구조화하여 제공하는 역할을 합니다.  
이미지, 텍스트, 오디오 등 다양한 형태의 데이터를 하나의 통일된 인터페이스로 다룰 수 있게 해줍니다.  

<br><br>

### 🟡 **DataLoader (데이터로더)**  

- **정의:**  
    - **"어떻게"** 학습 데이터를 효율적으로 모델에 전달할 것인가에 대한 **방법**을 정의  
    - `Dataset`에서 데이터를 **배치(batch) 단위로 묶어 모델에 공급해주는 역할**을 하는 클래스.  
    - `DataLoader`는 `Dataset` 객체를 입력으로 받으며, 학습 과정에서 데이터를 효율적으로 불러올 수 있도록 도와줌.  

- `DataLoader`의 주요 기능  
    - **배치 처리 (Batching):** 전체 데이터를 한 번에 메모리에 올리는 것이 아니라, 지정된 `batch_size`만큼의 데이터를 묶어서 모델에 전달합니다. 이는 GPU 메모리 효율성을 극대화하고, 병렬 연산을 가능하게 하여 학습 속도를 높여줍니다.  
    - **데이터 셔플링 (Shuffling):** 매 에폭(epoch)마다 데이터셋의 순서를 무작위로 섞어줍니다. 이는 모델이 데이터의 순서에 의존하여 학습하는 것을 방지하고, 일반화 성능을 향상시키는 데 도움을 줍니다.  
    - **병렬 로딩 (Parallel Loading):** `num_workers` 인자를 통해 여러 개의 서브프로세스를 사용하여 데이터를 미리 로드해 놓습니다. 이는 CPU가 데이터를 준비하는 동안 GPU가 학습 연산을 수행하게 하여, 데이터 로딩으로 인한 병목 현상을 줄여줍니다.  

<br>

#### **즉, `Dataset`은 원본 데이터를 구조화하고, `DataLoader`는 이 구조화된 데이터로부터 필요한 만큼을 가져와 모델이 학습하기 좋은 형태로 제공하는 역할을 수행합니다.**  

<br><br>

### 🟡 간단한 예제 살펴보기  

In [ ]:
from torch.utils.data import Dataset, DataLoader


# Dataset = 자판기 (음료수 0~9 들어 있음)
class MyDataset(Dataset):
    def __init__(self):
        self.data = list(range(9))  # [0,1,2,3,4,5,6,7,8,9]

    def __len__(self):
        return len(self.data)  # 총 10개

    def __getitem__(self, idx):
        return self.data[idx]  # idx 번째 음료수 꺼내기


# 자판기 준비
dataset = MyDataset()


# DataLoader = 자판기 팔
# class DataLoader:
#     def __init__(self, dataset, batch_size=1, shuffle=False):
#         self.dataset = dataset  # 자판기
#         self.batch_size = batch_size  # 한 번에 꺼낼 개수
#         self.shuffle = shuffle  # 섞을지 말지
loader = DataLoader(dataset, batch_size=3, shuffle=True)


for batch in loader:
    print(batch)

tensor([2, 0, 8])
tensor([6, 1, 3])
tensor([7, 5, 4])


In [3]:
from torch.utils.data import Dataset, DataLoader    

# ----------------------------------------------------------------
# 1. Dataset 만들기: 창고 만들기
# 여기서는 간단한 가상의 데이터셋을 만듭니다.
# 100개의 숫자(데이터)와 그에 대한 100개의 정답(레이블)이 있다고 가정합니다.
class SimpleDataset(Dataset):
    def __len__(self):
        return 100  # 창고에 총 100개의 재료가 있다고 알려줍니다.

    def __getitem__(self, idx):
        # index번째 재료를 꺼내는 법
        data = idx  # 'idx'번째 재료(데이터)와 정답(레이블)을 반환합니다.
        label = idx * 2  # 데이터는 idx, 정답은 idx*2라고 가정합니다.
        return data, label

# ----------------------------------------------------------------
# 2. DataLoader 만들기: 카트에 담아주는 도우미 고용하기
dataset = SimpleDataset()   # 창고(SimpleDataset)를 넘겨주고, 
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)  # 한 번에 10개씩(batch_size=10) 담아달라고 요청.

# ----------------------------------------------------------------
# 3. 모델 학습 과정처럼 데이터 사용해 보기
# 이제 이 'dataloader'만 사용해서 학습을 시작할 수 있습니다.
for i, (batch_data, batch_labels) in enumerate(dataloader):
    print(f"[{i+1}번째 배치] 데이터: {batch_data.tolist()}, 정답: {batch_labels.tolist()}")
    if i == 2:
        break # 예시를 위해 3번만 출력합니다.

[1번째 배치] 데이터: [26, 13, 71, 40, 20, 47, 18, 58, 17, 77], 정답: [52, 26, 142, 80, 40, 94, 36, 116, 34, 154]
[2번째 배치] 데이터: [23, 48, 61, 42, 56, 49, 0, 72, 99, 4], 정답: [46, 96, 122, 84, 112, 98, 0, 144, 198, 8]
[3번째 배치] 데이터: [66, 74, 28, 36, 8, 12, 15, 33, 62, 41], 정답: [132, 148, 56, 72, 16, 24, 30, 66, 124, 82]
